In [1]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd

In [127]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgress")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [128]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432")

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
s = 250                                            # desired sample size
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT * FROM fakenews.article", connection)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df.head()

,id,domain_id,type_id,url,content,title,meta_description,scraped_at,updated_at,inserted_at
0,13842,1,1,http://beforeitsnews.com/military/2017/12/russ...,"['russia', 'responds', 'to', 'us', 'provocatio...",Russia Responds to US Provocation: Open Skies ...,[''],2018-01-25,2018-02-02,2018-02-02
1,2350,2,2,http://breakpoint.org/category/signs-and-wonde...,"['signs', 'and', 'wondersr', 'r', 'snap', 'out...",Signs and Wonders Archives,[''],2018-01-25,2018-02-02,2018-02-02
2,27373,3,3,https://www.undergroundhealth.com/understandin...,"['by', 'giovani', '', '', 'URL', 'r', 'r', 'ar...",Understanding What Authentic Happiness Is,[''],2018-01-25,2018-02-02,2018-02-02
3,26536,0,0,https://www.alternet.org/activism/if-we-dont-a...,"['if', 'we', 'dont', 'act', 'now', 'fascism', ...","If We Don't Act Now, Fascism Will Be on Our Do...",[''],2018-01-25,2018-02-02,2018-02-02
4,6045,1,1,http://beforeitsnews.com/opinion-conservative/...,"['greg', 'hunter', 'big', 'banks', 'in', 'big'...","Greg Hunter: Big Banks in Big Trouble, Syria/N...",[''],2018-01-25,2018-02-02,2018-02-02


In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

df['content'] = df['content'].apply(' '.join)

x = v.fit_transform(df["content"])

In [131]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df["type_id"], test_size=0.8, random_state=0)

In [133]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(11, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [157]:
model.fit(X_train, y_train.values, epochs=3)

Train on 50 samples
Epoch 1/3
50/50 [==============================] - 0s 239us/sample - loss: 2.3750 - accuracy: 0.5400
Epoch 2/3
50/50 [==============================] - 0s 120us/sample - loss: 2.3686 - accuracy: 0.5000
Epoch 3/3
50/50 [==============================] - 0s 139us/sample - loss: 2.3618 - accuracy: 0.4000


In [158]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
val_loss, val_acc = model.evaluate(X_test,y_test.values)
print(val_loss, val_acc)

200/200 [==============================] - 0s 90us/sample - loss: 2.3733 - accuracy: 0.4550
2.373314743041992 0.455
